In [8]:
from pydicom import dcmread
from glob import glob
from torch.utils.data import Dataset, DataLoader
from skimage import transform
import sklearn.preprocessing as skp
import numpy as np
import torch
from typing import Tuple

In [9]:
data_path = "/Users/abinavr/Desktop/mri_data/train/**/**/T1postcontrast"
label_path = "/Users/abinavr/Desktop/mri_data/train/**/**/T2SPACE"

In [10]:
def to_tensor(array):
    """
    input image array of form WxHxD
    output tensor of the form DxHxW
    """
    img = np.transpose(array, (2, 0, 1))
    img = torch.from_numpy(img)
    return img


def normalize(array):
    """
    Input image is given as a 2D array
    Resizing is done with sklearn.preprocessing.normalize
    """
    image = skp.normalize(array[:, :, 0])
    return image

def resize(array, size):
    """Resizes a given array into size shape

    Args:
        array (numpy.ndarray): Image to be resized
        size (Tuple): Size of the output image
    """
    image = transform.resize(array, size)
    return image

In [11]:
class ImageLoader(Dataset):
    def __init__(self, dataset_path: str, label_path: str, image_size: int) -> None:
        # super(ImageLoader, self).__init__()
        self.dataset = glob(f"{dataset_path}/*.dcm")
        self.labels = glob(f"{label_path}/*.dcm")
        self.image_size: Tuple = (image_size, image_size)

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, index):
        try:
            pic = dcmread(self.dataset[index]).pixel_array
            label = dcmread(self.labels[index]).pixel_array
            resized_pic = resize(pic, self.image_size)
            resized_label = resize(label, self.image_size)
            normalized_pic = np.expand_dims(resized_pic, axis=-1)
            label = np.expand_dims(resized_label, axis=-1)
            data = to_tensor(normalized_pic)
            label = to_tensor(label)
            return data, label
        except Exception as e:
            print(f"Error loading data at index {index}: {str(e)}")
            return None, None

def loader_instance(dataset_path, label_path, image_size, batch_size, num_workers):
    dataset = ImageLoader(dataset_path=dataset_path, label_path=label_path, image_size=image_size)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return loader

In [12]:
train_dataloader = loader_instance(data_path, label_path, image_size=128, batch_size=4, num_workers=0)

In [13]:
len(train_dataloader)

3549

In [14]:
for data,label in train_dataloader:
    print(data.shape, label.shape)

torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128])
torch.Size([4,

TypeError: expected Tensor as element 1 in argument 0, but got NoneType